In [6]:
"""
les symboles particulières comme ? ! et . sont supprimé dans la question.
Il faut peut être returner une réponse spécifique si il rencontre plus 1 mot qu'il existe pas dans le corpus
Nous suprimmons également:
  1. Mot qui contient une seule lettre(qui sont suivante des erreurs)
  2. Une fonction unique pour nettoyage tous les données dans train-test et chat
Nous allons traiter tous les mots clés suivant :
km, vitesse, carburant, huile, position, batterie, pression_pneu. Il reste trajet et vin

Méthode TF_IDF donne un résultat 72.8% on train et 53.7% on test, qui est un résultat faible. Nous 
cherchons une autre méthode pour espérer un meilleur résultat...
  
"""
#import sys
#!{sys.executable} -m pip uninstall gtts

#from gtts import gTTS
#from playsound import playsound
#import speech_recognition
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('french')
from nltk.stem import *
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french", ignore_stopwords = False)
import csv
import re
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import math
import time
from dateparser.search import search_dates
import pandas as pd
import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
import numpy as np
from scipy.stats import entropy

def date_time_():
    
    DICT_NUMERO = {'ce':1, 'cette':1, 'un':1, 'une':1, 'deux':2, 'trois':3,\
                   'quatre':4, 'cinq':5, 'six':6, 'sept':7,\
              'huit':8, 'neuf':9, 'dix':10, 'onze':11, 'douze':12, 'treize':13,'quartoze':14,\
               'quinze':15, 'seize':16, 'dix-sept':17,'dix sept':17, 'dix huit':18, \
               'dix-huit':18, 'dix-neuf':19, 'vingt':20, 'ving et un':21, 'trente':30, 'quarante':40,\
                  'quarante cinq':45, 'soixante':60}
    DICT_TEMPS = {'heure':1, 'matin':4, 'après-midi':2,  'journée':24,  'jour':24, \
              'mois':30*24, 'semaine':7*24, \
              'an':24*365, 'année':24*365, 'minute': 1/60}
    DICT_COMPLET_TEMPS = {}
    for temps in DICT_TEMPS:
        for numéro in DICT_NUMERO:
            key1 = numéro + ' '+temps
            key2 = str(DICT_NUMERO[numéro]) + ' ' +temps
            value = DICT_NUMERO[numéro]*DICT_TEMPS[temps]
            DICT_COMPLET_TEMPS[key1] = value
            DICT_COMPLET_TEMPS[key2] = value
    DICT_FOR_SPECIAL_CASE = {'aujourd\'hui':8, 'hébdomadaire':7*24, 'annuel': 365*24, \
                           'hier': 24, 'avant-hier': 48, 'annuelle': 365*24 , 'actuellement':1/2,\
                            'maintenant':1/2}
    DICT_COMPLET_TEMPS['#cas_particulier#'] = DICT_FOR_SPECIAL_CASE
    DICT_COMPLET_TEMPS['avant hier'] = 72
    DICT_COMPLET_TEMPS['semaine dernière'] = 24*7
    return DICT_COMPLET_TEMPS

def construct_list_city():
    #import csv
    liste_ville =[]
    with open('data/villes_france.csv') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            liste_ville.append(row[2])
        for ville in liste_ville:
            if len(ville) <4:
                liste_ville.remove(ville)
    return liste_ville

def _find_right_time(question, Dict):
    
    list_ = ['?', '.', ' ', '\n', '!']
    for symbol in list_:
        question = question.lower().strip(symbol)
    list_word = question.split()
    word_ = None
    list_ = []
    if len(list_word) < 2:
        return None, None 
    for word in list_word:
        if word_ is not None:
            list_.append(str(word_)+' '+str(word))  
        word_ = word
    for key in Dict:
        for word in  list_:
            if key == word:
                return Dict[key], key
    for word in Dict['#cas_particulier#']:
        for word2 in question.split():
            if word2 == word:
                return Dict['#cas_particulier#'][word], word       
    return None, None


def _find_right_geography(question, Dict):
    """
    Nous cherchons une ville, un département dans le dictionnaire pour retourner soit
    ce nom de ce lieu, soit ses coordonnées correspondance. Le retour de ce donnée dépend 
    de quel façon nous allons traiter avec cette information.
    
    """
    list_ = ['?', '.', '\n', '!', ',']
    for symbol in list_:
        question = question.replace(symbol, '') 
    list_geography = []
    for word in question.split():
        if word.lower() !=word:
            word1 = word.lower()
            if word1 in Dict:
                list_geography.append(word)
    if len(list_geography) >0:
        return list_geography
    else:
        return None

def list_variable():
    """
    list_var is to manually complete """
    list_var = []
    if len(list_var) >0:
        return list_var
    else:
        return None

def _normalized_table():
    """
    return a data frame with new name
    """
    import pandas as pd
    df = pd.read_csv('fleet_donnees.csv', sep = ';')
    variable_names = list_variable()
    if variable_names ==None:
        return df
    else:
        name = list(df)
        dict_name = {name[i]: variable_names[i] for i in range(len(name))}
        df = df.rename(index= str, columns = dict_name)
        return df

def find_right_time_from_table(time, df):
    """
    given time found from a question, match it with the time given in table
    This depend on format of time in table.
    
    """
    return None
    
    
def clearing_word(word):
    word = re.sub('\x8e', 'é', word)
    word = re.sub('\x88', 'à', word)
    word = re.sub('\x9d', 'ù', word)
    word = re.sub('\x8f', 'è', word)
    word = re.sub('\x9e', 'û', word)
    word = re.sub('\x90', 'ê', word)
    word = re.sub('\x99', 'ô', word)
    word = re.sub('\x94', 'î', word)
   # word = re.sub('\x8f', 'è', word)
    word = re.sub('\x8d', 'ç', word)
    word = re.sub('õ', '', word)
    word = re.sub('Ê', '', word)
    word = re.sub('[?,.,!, \,,  %]', '', word)
    if word == 'û' or word == 'v' or word == 'é':
        word = ''
    if word =="2017êles":
        word = "2017"
    if "ênox" in word:
        word ="nox"
    return word


def clear_line(pharse, stop_word):
    """
    Arg: string
    Return: string
    """
    dict_manuel = dict_manuelle()
    pharse_ =[]
    for words in pharse.strip().strip('?').strip('.').strip('!').lower().split(' '):
        for word in words.split('-'):
            for word_ in word.split('\''):
                word_ = clearing_word(word_)
                word_ = stemmer.stem(word_)
        if word_ not in stop_word and len(word_) >1:       
            pharse_.append(word_)  
        for i in range(len(pharse_)):
            for line in dict_manuel:
                if pharse_[i] in line:
                    pharse_[i] = line[0]
    return ' '.join(pharse_)

def stemmer_line(line):
    return ' '.join([stemmer.stem(word) for word in line.split()])

def prepareData(PAIRS):
    pairs_trains, pairs_tests = [], []
   # PAIRS.extend(pairs1)
    index_train = random.sample(range(len(PAIRS)), int(len(PAIRS)*0.80))
    for i in range(len(PAIRS)):
        if i in index_train:
            pairs_trains.append(PAIRS[i])
        else:
            pairs_tests.append(PAIRS[i])
    print("Read %s sentence pairs of training set" % len(pairs_trains))
    print("Read %s sentence pairs of test set" % len(pairs_tests))
    return pairs_trains, pairs_tests

def get_all_convos(stopwords, file):
    
    convos = []
    questions_to_print = []
    liste_file = [file]
    list_of_word = []
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = str(line)
                    if '++++' in question:
                         question = question[8:]
                    question_ = question.strip().strip('!').strip('?').strip('.')
                    questions_to_print.append(question_)
                    question = clear_line(question, stopwords)
                else:
                    answer = str(line)
                    if '++++' in answer:
                        answer = answer[8:].strip()
                    convos.append([question, answer])
                i+=1
        f.close()
        good_index = []
        convos_ = []
        for i in range(len(convos)):
            if convos[i][0] not in convos_:
                convos_.append(convos[i][0])
                good_index.append(i)
            
    
    return [convos[i] for i in good_index], [questions_to_print[i] for i in good_index]




def get_all_questions_to_print(file):
    
    convos = []
    liste_file = [file]
    list_of_word = []
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = str(line)
                    if '++++' in question:
                         question = question[8:]
                    question = question.strip()
                    convos.append(question)
                i+=1
        f.close()
        indexes = []
        index = 0
    for question in convos:
        question = clear_line(question, STOPWORDS)
        if len(question) !=0:
                indexes.append(index)
        index +=1
    return [convos[i] for i in indexes]

def get_list_words(convos):
    list_words =[]
    for pair in convos:
        for word in pair[0].split():
            if word not in list_words:
                list_words.append(word)
    print('There are {} words in corpus:'.format(len(list_words)))
    return list_words
                

def simulation_pairs():
    
    DIRECT_VARIABLE = ['vitesse', 'batterie', 'position', 'km']
    BORDE = {'MAX': ['plus grand', 'plus grande', 'plus grands', 
         'plus grandes',
         'plus vite', 'max', 'maximum', 'maximal', 'maximaux', 'maximale',
        'plus haut', 'plus haute', 'plus hautes', 'plus hauts',  
        'plus élevé', 'plus élevée', 'plus élevés', 'plus élevées'], 
         'MIN':['moins grand', 'moins grande', 'moins grands','moins grandes',
        'plus petit', 'plus petite', 'plus petits', 'plus petites','plus faible', 'plus faibles', 
         'min', 'minimal', 'minimale', 'minimales', 'minimaux','moins vite', 
         'moins élevé', 'moins élevée', 'moins élevés', 'moins élevées'],
        'MOYENNE': ['moyenne', 'moyen', 'moyennement']
        }
    COMPLEX_ANALYSIS = ['problème', 'problèmes', 'erreurs', 'erreur', 'danger']
    PAIRS = []
    question_for_simulation = ['quels véhicules ont', 'quel véhicule', 'quelle voiture', 'quelles voitures']
    reponse_for_simulation = 'véhicule'
    for borde in BORDE:
        for word in BORDE[borde]:
            for direct_variable in DIRECT_VARIABLE:
                for question_f_s in question_for_simulation:
                    if direct_variable !='position':
                        question= question_f_s+' '+direct_variable + ' '+ word
                        key0 = '#id#'
                        key1 = '#'+direct_variable+'#'
                        key2 = '#'+borde +'#'
                        reponse = reponse_for_simulation +' '+key0+ ' '+ key1+ ' '+ key2
                        PAIRS.append([question,reponse])

    question_for_simulation = ['', 'quel véhicule a', 'quelle voiture a', 'quelles voitures ont', 'il y a']
    reponse_for_simulation = 'véhicule' 
    for direct_variable in DIRECT_VARIABLE:
        for complex_analysis in COMPLEX_ANALYSIS:
            for question_f_s in question_for_simulation:
                #if direct_variable !='position':
                      question= question_f_s+' '+complex_analysis + ' '+ direct_variable
                      key0 = '#id#'
                      key1 = '#'+direct_variable+'#'
                      key2 = '#'+ complex_analysis +'#'
                      reponse = reponse_for_simulation +' '+key0+ ' '+ key2 +' '+ key1
                      PAIRS.append([question,reponse])
    return PAIRS

def clear_pairs_trains(pairs_trains):
    questions = []
    index = []
    for i in range(len(pairs_trains)):
        if  pairs_trains[i][0] not in questions:
            index.append(i)
            questions.append(pairs_trains[i][0])
    return [pairs_trains[i] for i in index]

def tf_idf(word, doc, méthode):
    tf, df = 0, 0
    len_doc = len(doc.split())
    value_doc = 1/math.sqrt(len_doc)
    value_doc = math.sqrt(value_doc)
    if méthode !='Lucene':
        value_doc = 1
    if word in KEY_LIST:
        value_word = 10*value_doc
    else:
        value_word = 1*value_doc
    if word in str(doc):
        tf = value_word
    for doc_ex in QUESTION_TO_TRAINS:
        if word in str(doc_ex):
            df+=1
    if méthode  == 'normal':
        if df>0:
            return tf*math.log(len(QUESTION_TO_TRAINS)/(df), 10)
        else:
            return 0
    if méthode == 'probabiliste':
        if df>0:
            return tf*math.log((len(QUESTION_TO_TRAINS)-df)/(df), 10)
        else:
            return 0
    if méthode == 'lissé':
        if df>0:
            return tf*(1+math.log(len(QUESTION_TO_TRAINS)/df, 10))
        else:
            return tf
    if méthode == 'probabiliste_lissé':
        if df>0:
            return tf*(1+math.log((len(QUESTION_TO_TRAINS)-df)/(df), 10))
        else:
            return tf

def dict_doc_score(doc, méthode = 'normal'): 
    """
    We return score of all one words and all two consequent words"""

    list_word_ = split_words(doc)    
    return {word: tf_idf(word, doc, méthode) for word in list_word_}
    
def dict_doc_score_for_classif(doc, sujet): 
    """
    We return score of all one words and all two consequent words"""

    list_word_ = split_words(doc)    
    return {word: tf_idf_for_classif(word, doc, sujet) for word in list_word_}

   
def split_words(doc):
    
    list1 = doc.split()
    list2 = []
    for word in list1:
        list2.append(word)
    for i in range(len(list1)-1):
        word = list1[i]+ ' '+list1[i+1]
        list2.append(word)
    if len(list1)>2:
        for i in range(len(list1)-2):
            word = list1[i]+' '+list1[i+1]+' '+list1[i+2]
            list2.append(word)
    return list2
        
        
def get_key_list(pairs_trains):
    
    dict_ = dict_manuelle()
    KEY_LIST = []
    KEY_LIST_ = []
    for pair in pairs_trains:
        for word in pair[1].split():
            if '#' in word:
                word = re.sub('#', '', word)
                if word not in KEY_LIST:
                    KEY_LIST.append(word)
    for key in KEY_LIST:
        key = stemmer.stem(key)
        if key not in KEY_LIST_ and '_' not in key:
            KEY_LIST_.append(key)
    KEY_RETURN = []
    for key in KEY_LIST_:
        boolean = False
        for list_ in dict_:
            if key in list_:
                KEY_RETURN.append(list_[0])
                boolean = True
                break
        if not boolean:
            KEY_RETURN.append(key)
    #for list_ in dict_:
       # if list_[0] not in KEY_RETURN:
        #    KEY_RETURN.append(list_[0])       
    return KEY_RETURN
        

def find_right_index(new_doc,DICT, méthode = 'normal'):
    
    index=0
    biggest_score = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[0],DICT, méthode)
    for i in range(len(QUESTION_TO_TRAINS)):
        boolean1 = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i], DICT, méthode) >biggest_score
        boolean2 = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i], DICT, méthode) == biggest_score
        boolean3 = len(QUESTION_TO_TRAINS[index]) >len(QUESTION_TO_TRAINS[i])
        if boolean1 or (boolean2 and boolean3):
            biggest_score = score_by_new_doc(new_doc, QUESTION_TO_TRAINS[i],DICT, méthode)
            index = i
    précision, rappel = precision_recall(new_doc, QUESTION_TO_TRAINS[index])
    #if biggest_score ==0 or précision <0.2 or rappel<0.2:
     #   for question in QUESTION_TO_TRAINS:
      #      if '_return_' in question:
       #         try:
        #            index = QUESTION_TO_TRAINS.index(question)
         #       except ValueError:
          #          index = -1
    return index

def reformule_question_if_need(new_question, good_question):
    
    if 'to_return' in good_question:
        return True
    score1 = score_by_new_doc(new_question, good_question, DICT)
    score2 = 0
    for word in Dict_of_methode[good_question]:
        score2 += Dict_of_methode[good_question][word]
    if score1/score2 > 0.6:
        return  True
    else:
        return  False


def test(DICT, méthode, train =False):
    
    if not train:
        index_to_print = random.sample(range(len(pairs_tests)), 20)
        #loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            #loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            #loss_total +=loss
            if i in index_to_print:
                print(pairs_tests[i])
                print(pairs_trains[index][1])
        print('-'*80)
       # print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        #loss_total = 0
        #index_to_print = random.sample(range(len(pairs_trains)), 20)  
        list_of_bad_prediction = []
        i0 = 0
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            #loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            #loss_total +=loss
            if index != i :
                i0 +=1
                print(pairs_trains[i])
                print(pairs_trains[index][1])
        print('-'*80)
        #print('ACCURACY=', 1-loss_total/len(pairs_trains))
        print('Il y a {} erreurs d\'indexes parmis {} prédictions'.format(i0, len(pairs_trains)))

        
def test_without_print(DICT, méthode, train =False):
    
    if not train:
        loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        loss_total = 0
        index_to_print = random.sample(range(len(pairs_trains)), 20)   
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_trains))

def change_subjet_of_question(question):
    dict_ = {'ton':'mon', 'ta':'ma', 'votre':'mon', 'tes':'mes', 'mon':'ton', \
            'ma':'ta', 'mes':'tes', 'tu': 'je', 'moi':'toi', 'je':'tu', 'toi':'moi',\
            'vous':'je'}
    question_ = []
    for word in question.lower().split():
        if word in dict_:
            word = dict_[word]
        question_.append(word)
    return ' '.join(question_)
        
def chat(méthode='normal'):
    
    print('Bonjour, C\'est le bot d\'Avicen, pose tes questions, s\'il te plaît!')
    path = 'processed/chat_tfidf.txt'
    f =  open(path, 'a+') 
    while True:
            line = str(input('Vous: '))
            list_city = _find_right_geography(line, CITY_LIST)
            line = line.lower().strip().strip('?').strip('.')
            time, time_word = _find_right_time(line, COMPLET_DICT_TIME)
            if time_word is not None:
                line = line.replace(time_word, '')
            if list_city is not None:
                for city in list_city:
                    line = line.replace(city, '')
            LINE = clear_line(line, STOPWORDS)
            unknown_word =0
            for word in LINE.split():
                if word not in LIST_OF_WORDS:
                    unknown_word += 1
            #if unknown_word > 1:
             #   print('Désolé, je ne comprends pas ta question, peux tu la reformuler s\'il te plaît?')
              #  continue
            if len(line) !=0:
                index= find_right_index(LINE, DICT, méthode)
                good_question = QUESTION_TO_TRAINS[index]
                if not reformule_question_if_need(LINE, good_question):
                    get_answer = str(input('Tu veux demander: {}?  |'.format(questions_sujet[index])))
                    if 'non' in get_answer.lower():
                        print('Peux tu reformuler ta question s\'il te plaît?')
                        continue
                if time == None and list_city == None:
                     print('bot: ', pairs_trains[index][1])
                elif list_city ==None:
                    print('Bot: {}, {}'.format(time_word, pairs_trains[index][1]))
                elif time == None:
                    cities = ' '.join(list_city)
                    print('Bot: À {}, {}'.format(cities, pairs_trains[index][1]))
                else:
                    cities = ' '.join(list_city)
                    print('Bot: À {}, {}, {}'.format(cities, time_word, pairs_trains[index][1]))
                f.write('VOUS ++++ '+line+'\n')
                f.write('BOT ++++ '+pairs_trains[index][1]+'\n')
            
            else:
                f.close()
                break 

def precision_recall(lstcomp, lstref):
    
    card_intersec = 0.0 # force à utiliser la division non entière
    for t in set(lstcomp) :
        card_intersec += min(lstref.count(t), lstcomp.count(t))
    if len(lstcomp)==0:
        precision =1
    else:
        precision = card_intersec/len(lstcomp)
    if len(lstref)==0:
        rappel = 1
    else:
        rappel = card_intersec/len(lstref)
    return (precision, rappel)

def answer_from_index(index, sujet):
    if sujet:
        convos = convos_sujet
    else:
        convos = convos_hors
    return convos[index][1]

def question_from_index(index, sujet):
    if sujet:
        convos = convos_sujet
    else:
        convos = convos_hors
    return convos[index][0]


               
def dict_of_all_score(METHODE):
    """
    return score of all word in all doc"""
    DICT = {}
    for méthode in METHODE:
        dict_for_this_method ={}
        for doc in QUESTION_TO_TRAINS:
            dict_for_this_method[doc]=dict_doc_score(doc, méthode)
        DICT[méthode] = dict_for_this_method
    return DICT
 
    
def chatting_with_corpus():

    path = 'processed/chat_tfidf.txt'
    i = 0
    questions = []
    with open(path, 'r') as f:
        for line in f:
            if i%2 ==0:
                line = line[9:]
                line = line.strip()
                questions.append(line)
            i +=1
    f.close()
    good_index = []
    questions_ = []
    index = 0
    for question in questions:
        question = clear_line(question, STOPWORDS)
        if question not in questions_:
            questions_.append(question)
            good_index.append(index)
        index += 1
    questions_clean = [questions[i] for i in good_index]
    for line in questions_clean:
            print('Humaine: ', line)
            time, time_word = _find_right_time(line, COMPLET_DICT_TIME)
            list_city = _find_right_geography(line, CITY_LIST)
            if time_word is not None:
                line = line.replace(time_word, '')
            if list_city is not None:
                for city in list_city:
                    line = line.replace(city, '')
            LINE = clear_line(line, STOPWORDS)
            unknown_word =0
            for word in LINE.split():
                if word not in LIST_OF_WORDS:
                    unknown_word += 1
            if unknown_word > 1:
                print('Désolé, je ne comprends pas ta question, peux tu la reformuler s\'il te plaît?')
                print()
                continue
            if len(line) !=0:
                index= find_right_index(LINE, DICT, méthode)
                if time == None and list_city == None:
                     print('Bot: ', pairs_trains[index][1])
                elif list_city ==None:
                    print('{}, {}'.format(time_word, pairs_trains[index][1]))
                elif time == None:
                    cities = ' '.join(list_city)
                    print('À {}, {}'.format(cities, pairs_trains[index][1]))
                else:
                    cities = ' '.join(list_city)
                    print('À {}, {}, {}'.format(cities, time_word, pairs_trains[index][1]))
            print()
               
def dict_manuelle():
    """
    return la liste des listes des synonymes"""
    list_ = []
    dict_ = {}
    dict_['voiture'] = ['voiture', 'véhicule']
    dict_['km'] = ['km', 'kilomètrage', 'kilométrage', 'distance', 'kilomètre']
    dict_['essence'] = ['essence', 'carburant', 'énergie']
    dict_['roule'] = ['roule', 'roulent', 'marche', 'marcher', 'circule', 'circulent', 'circulation']
    dict_['bien'] = ['bien', 'bonne', 'good', 'excellent', 'parfait', 'beau', 'ok']
    dict_['min']  = ['min', 'minimum', 'minimal']
    dict_['pneu'] = ['pneumatique', 'pneu', 'pneus']
    dict_['pouquoi'] = ['cause', 'pourquoi', 'raison', 'motif']
    dict_['stupid'] = ['stupid', 'con', 'idiot', 'bête', 'imbécile']
    dict_['connaître'] = ['connaître', 'connaitre', 'savoir', 'comprendre',\
                          'connaisance', 'connais', 'connaîs', 'sais', 'savez']
    dict_['problème']  = ['problème', 'difficulté', 'danger', 'ennui', 'dangers']
    dict_['erreur'] = ['faute', 'erreur']
    dict_['arrêt'] = ['arrêt', 'arrêter', 'stopper', 'immobilisé', 'immobiliser', 'immobile', 'paralisé']
    dict_['disponible'] = ['disponible', 'disponibilité', 'libre']
    dict_['abimer'] = ['abîmer', 'hors service', 'cassé', 'endommager', 'abîmé']
    dict_['boitier'] = ['boitier', 'boîtier', 'boîte']
    dict_['frein'] = ['frein', 'freinage', 'freiner']
    dict_['parcouru'] = ['parcouru', 'parcourus', 'parcourir']
    dict_['peux'] = ['peux', 'pouvoir', 'pourrais']
    dict_['veux'] = ['veux', 'vouloir', 'veut', 'voulais', 'voudrais', 'souhaiter', 'souhaite']
    dict_['fait'] = ['fait', 'fais', 'faire', 'faites']
    dict_['fort'] = ['fort', 'puissant', 'robuste', 'solide']
    dict_['grand'] = ['grand', 'gross', 'élevé', 'haut', 'haute']
    dict_['mal'] = ['mal', 'mauvais', 'souci']
    dict_['changer'] = ['modifier', 'modification', 'changer', 'changement']
    dict_['avoir'] = ['as', 'obtenir', 'avoir', 'posséder', 'disposer']
    dict_['peur']  = ['peur', 'craint']
    dict_['placer'] = ['placer','installer']
    dict_['nombre'] = ['nombre', 'combien', 'quantité']
    dict_['aider']  = ['aider', 'aide', 'soutien', 'soutenir']
    dict_['question'] = ['question', 'demande']
    dict_['information'] = ['information', 'infos', 'renseignement', 'indictation',\
                            'informer', 'renseigner','indiquer']
    dict_['usé'] = ['usé', 'usure', 'vieux', 'vieille', 'fatigué']
    dict_['réduire']  = ['réduire', 'réduction', 'abaisser', 'diminuer', 'diminution']
    dict_['présent']  = ['présent', 'maitenant', 'actuel', 'actuellement']
    dict_['suivre']   = ['suivre', 'observer', 'surveiller', 'poursuivre']
    dict_['aller']    = ['aller', 'vas', 'vais', 'va']
    dict_['échanger'] = ['échanger', 'estimer', 'échangement', 'estimation',\
                         'évaluer', 'calculer','évaluation'\
                        , 'mesurer']
    dict_['taux']    = ['taux', 'pourcentage']
    dict_['ampoule'] = ['ampoule', 'feux', 'feu']
    for key in dict_:
        l = []
        for word in dict_[key]:
            l.append(stemmer.stem(word))
        list_.append(l)
    return list_

    
                
def chatting_with_test_corpus():
    
    path = 'test.txt'
    questions = []
    with open(path, 'r') as f:
        for line in f:
                line = line.strip()
                questions.append(line)
    f.close()
    questions_clean = []
    for question in questions:
        if question not in questions_clean:
            questions_clean.append(question)
    for line in questions_clean:
            line1 = line
            time, time_word = _find_right_time(line, COMPLET_DICT_TIME)
            list_city = _find_right_geography(line, CITY_LIST)
            if time_word is not None:
                line = line.replace(time_word, '')
            if list_city is not None:
                for city in list_city:
                    line = line.replace(city, '')
            LINE = clear_line(line, STOPWORDS)
            unknown_word =0
            print('Vous: ', line1)
            for word in LINE.split():
                if word not in LIST_OF_WORDS:
                    unknown_word += 1
            if unknown_word > 1:
                print('Bot: Désolé, je ne comprends pas ta question, peux tu la reformuler s\'il te plaît?')
                print()
                continue
            if len(line) !=0:
                index= find_right_index(LINE, DICT, méthode)
                if time == None and list_city == None:
                     print('Bot: ', pairs_trains[index][1])
                elif list_city ==None:
                    print('Bot: {}, {}'.format(time_word, pairs_trains[index][1]))
                elif time == None:
                    cities = ' '.join(list_city)
                    print('Bot: À {}, {}'.format(cities, pairs_trains[index][1]))
                else:
                    cities = ' '.join(list_city)
                    print('Bot: À {}, {}, {}'.format(cities, time_word, pairs_trains[index][1]))
               
                LINE = clear_line(line, STOPWORDS)   
                index = find_right_index(LINE, DICT, METHODE[0])
                #print('YOU: ', line)
                #print('BOT: ', pairs_trains[index][1])
                print()
            

def get_all_and_all_convos():
    convos = []
    file1 = 'data/convos27juin_test.txt'
    file2 = 'data/convos27juin_train.txt'
    files = [file1, file2]
    for file in files:
        with open(file, 'r') as f:
            i=0
            for line in f:
                if i%2==0:
                    question = line.strip()
                    if '++++' in question:
                         question = question[9:]
                else:
                    answer = line.strip()
                    if '++++' in answer:
                        answer = answer[9:]
                    convos.append([question, answer])
                i+=1
        f.close()
    return convos


def get_all_convos_in_movies():
    convos = []
    raw_questions = []
    raw_convos = []
    for root, dirs, files in os.walk("soustitre"):
        for file in files:
            if file.endswith(".txt"):
                path = os.path.join(root, file)
                i = 0
                with open(path, 'r') as f:
                    get_ = False
                    for line in f.readlines():
                        line = line.strip('-')
                        if i > 3:
                            if '?' in line and get_ == False:
                                question = line.strip()
                                get_ = True
                            else:
                                if get_ == True:
                                    answer = line.strip()
                                    convos.append([clear_line(question), answer])
                                    raw_convos.append([question, answer])
                                    raw_questions.append(question)
                                    get_ = False
                          
                        i +=1
    index_of_train = random.sample(range(len(convos)), int(.8*len(convos)))
    raw_questions = [raw_questions[i] for i in index_of_train]
    pairs_trains = [convos[i] for i in index_of_train]
    pairs_tests = [convos[i] for i in range(len(convos)) if i not in index_of_train]
    return pairs_trains, pairs_tests, raw_questions, raw_convos

def clear_all_convos(all_convos, name):
    """
    Supprimer tous les couples où les questions sont les mêmes"""
    index_of_convos = []
    question = []
    index =0
    for pair in all_convos:
        if pair[0] not in question:
            index_of_convos.append(index)
            question.append(pair[0])
        index +=1
    convos = [all_convos[index] for index in index_of_convos]
    print('Il y a {} de couples de  {}'.format(len(convos), name))
    return convos

def save_convos_in_file(all_convos):
    
    index_of_train = random.sample(range(len(all_convos)), int(len(all_convos)*0.80))
    with open('convos6juillet_train.txt', 'w') as f:
        for index in index_of_train:
            pair = all_convos[index]
            f.write('YOU ++++ '+ pair[0]+'\n')
            f.write('BOT ++++ '+ pair[1]+'\n')
    f.close()
    with open('convos6juillet_test.txt', 'w') as f:
        for index in range(len(all_convos)):
            if index not in index_of_train:
                pair = all_convos[index]
                f.write('YOU ++++ '+ pair[0]+'\n')
                f.write('BOT ++++ '+ pair[1]+'\n')
    f.close()
    len_test = len(all_convos)- len(index_of_train)
    print('save {} convos in train and {} convos in test'.format(len(index_of_train), len_test))
    
def test_to_find_error(dict):
    
    index_of_error_prediction = []
    index_of_true_pair = []
    i = 0
    for pair in pairs_trains:
        index = find_right_index(pair[0],dict,  'normal')
        if index != i:
            index_of_error_prediction.append(index)
            index_of_true_pair.append(i)
        i +=1
    for i in range(len(index_of_error_prediction)):
        index1 = index_of_error_prediction[i]
        index2 = index_of_true_pair[i]
        score1 = score_by_new_doc(pairs_trains[index1][0], pairs_trains[index2][0],dict, 'normal')
        score2 = score_by_new_doc(pairs_trains[index2][0], pairs_trains[index2][0],dict, 'normal')
        print('False return question--- {} ---with score {}'.format(pairs_trains[index1][0], score1))
        print('True return question--- {} ---with score {}'.format(pairs_trains[index2][0], score2))

def print_all_errors_on_test(dict):
    for pair in pairs_tests:
        question = clear_line(pair[0], STOPWORDS)
        index = find_right_index(question, dict, 'normal')
        print(pair)
        print(pairs_trains[index])
        print()
        



def mean_score(doc, dict_):
    if len(doc) == 0:
        return 0
    total_score = 0
    doc_ =  doc.lower().strip().strip('?').strip('.').strip('!').split()
    for pharse in dict_:
        score = 0
        for word in doc_:
            if word in dict_[pharse]:
                score +=dict_[pharse][word]
        total_score += score/len(doc_)
    return total_score/len(dict_)

def find_right_index_for_classif(q, questions,  dict_):

    index=0
    biggest_score = score_by_new_doc_(q, questions[0], dict_)
    for i in range(len(questions)):
        boolean1 = score_by_new_doc_(q, questions[i], dict_) >  biggest_score
        boolean2 = score_by_new_doc_(q, questions[i], dict_) == biggest_score
        boolean3 = len(questions[index]) >len(questions[i])
        if boolean1 or (boolean2 and boolean3):
            biggest_score = score_by_new_doc_(q, questions[i], dict_)
            index = i
    return index
    
    

def classification(question, dict_sujet_, dict_hors_):
    
    seuil = 0.75
    Q1, Q2 = get_questions_for_classification()
    index_sujet = find_right_index_for_classif(question, Q1, dict_sujet_)
    index_hors  = find_right_index_for_classif(question, Q2, dict_hors_)
    score_sujet = score_by_new_doc_(question, Q1[index_sujet], dict_sujet_)
    score_hors  = score_by_new_doc_(question, Q2[index_hors], dict_hors_)
    test = score_sujet > score_hors
    précision_sujet, rappel_sujet = precision_recall(question, Q1[index_sujet])
    précision_hors, rappel_hors = precision_recall(question, Q2[index_hors])
    if test and précision_sujet >= seuil and rappel_sujet >= seuil:
        return True
    elif not test and précision_hors >= seuil and rappel_hors >= seuil:
        return False
    else:
        if mean_score(question, dict_sujet_) > mean_score(question, dict_hors_):
            #return False
            return True
        else:
            return False

    
def test_classification_by_corpus(dict_sujet, dict_hors):
    

    path = 'processed/chat_tfidf.txt'
    i = 0
    questions = []
    with open(path, 'r') as f:
        for line in f:
            if i%2 ==0:
                if '++++' in line:
                    line = line[8:]
                line = line.strip('+').strip()
                questions.append(line)
            i +=1
    f.close()
    total_value = 0
    for question in questions:
        value = classification(question, dict_sujet, dict_hors)
        total_value +=value
        print(question,'---------',value)
    print('Le pourcentage des sujets sont {}'.format(total_value/len(questions)))

    
    
    
def question_processing(question):
    question_ = question.strip().split()
    if len(question_) == 1:
        return question
    else:
        index = random.sample(range(len(question_)), len(question_)-1)
    return ' '.join([question_[i] for i in sorted(index)])


def dict_of_all_score_(convos, KEY_LIST, sujet):
    """
    return score of all word in all doc"""
    DICT = {}
    for doc_ in convos:
        doc = doc_[0]
        DICT[doc]=dict_doc_score_(doc,KEY_LIST, sujet)
    return DICT 



def find_right_index_(new_doc, sujet):
    
    if sujet:
        DICT = dict_sujet
        convos = convos_sujet
    else:
        DICT = dict_hors
        convos = convos_hors
    index=0
    biggest_score = score_by_new_doc_(new_doc, convos[0][0],DICT)
    for i in range(len(convos)):
        boolean1 = score_by_new_doc_(new_doc, convos[i][0], DICT) > biggest_score
        boolean2 = score_by_new_doc_(new_doc, convos[i][0], DICT) == biggest_score
        boolean3 = len(convos[index][0]) >len(convos[i][0])
        if boolean1 or (boolean2 and boolean3):
            biggest_score = score_by_new_doc_(new_doc, convos[i][0], DICT)
            index = i
    return index


def tf_idf_(word, doc, KEY_LIST, sujet):
    
    if sujet:
        convos = convos_sujet
    else:
        convos = convos_hors
    
    tf, df = 0, 0
    words = split_words(doc)
    #len_doc = len(words)
    #value_doc = 1/math.sqrt(len_doc)
    #value_doc = math.sqrt(value_doc)
    if word in KEY_LIST:
        value_word = 10
    else:
        value_word = 1
    if word in str(doc):
        tf = value_word
    for doc_ in convos:
        if word in str(doc_[0]):
            df+=1
    if df>0:
            return tf*math.log(len(convos)/(df), 10)
    else:
            return 0
        
def tf_idf_for_classif(word, doc, sujet = True):
    
    convos1, convos2 = get_questions_for_classification()
    if sujet:
        convos = convos1
    else:
        convos = convos2
    
    tf, df = 0, 0
    if word in str(doc):
        tf = 1
    for doc_ in convos:
        if word in str(doc_):
            df+=1
    if df>0:
            return tf*math.log(len(convos)/(df), 10)
    else:
            return 0

def chat_with_each_subjet(raw_question, sujet = True):
    
    if sujet:
        DICT = dict_sujet
        convos = convos_sujet
    else:
        DICT = dict_hors
        convos = convos_hors
    list_city = _find_right_geography(raw_question, CITY_LIST)
    line = raw_question.lower().strip().strip('?').strip('.')
    time, time_word = _find_right_time(line, COMPLET_DICT_TIME)
    if time_word is not None:
                line = line.replace(time_word, '')
    if list_city is not None:
                for city in list_city:
                    line = line.replace(city, '')
    LINE = clear_line(line, STOPWORDS)
    if len(line) !=0:
            index= find_right_index_(LINE, sujet)
            good_question = convos[index][0]
           # if not reformule_question_if_need(LINE, good_question):
                   # get_answer = str(input('Tu veux demander: {}?  |'.format(questions_to_print[index])))
                   # if 'non' in get_answer.lower():
                    #    print('Peux tu reformuler ta question s\'il te plaît?')
                     #   continue
            if time == None and list_city == None:
                     answer= convos[index][1]
            elif list_city ==None:
                    answer = time_word + ' '+ convos[index][1]
            elif time == None:
                    cities = ' '.join(list_city)
                    answer = 'À ' + cities +' '+ convos[index][1]
            else:
                    cities = ' '.join(list_city)
                    answer = 'À ' + cities +' '+ time_word+ ' '+ convos[index][1]
            return answer, index, line
    else:
        return None, None, line
    

def request_missing_infos(new_question, good_question, dict_):
    
    if 'to_return' in good_question:
        return True
    score1 = score_by_new_doc_(new_question, good_question, dict_)
    score2 = 0
    for word in dict_[good_question]:
        score2 += dict_[good_question][word]
    if score1/score2 > 0.6:
        return  True
    else:
        return  False
    
def to_print_answer(line):
    line = str(line)
    first = line[0]
    line = first.upper()+ line[1:]
    if line[-1] not in ['?','.','!']:
        line = line+'.'
    line = line.strip(' ')
    return line
    
def chat_():
    
    print('This will be LDA method....')
    print('Bonjour, le bot d\'Avicen à ton écoute')
    path = 'processed/chat_tfidf.txt'
    f =  open(path, 'a+') 
    while True:
            raw_question = str(input('You: '))
            f.write('YOU++++ '+ raw_question + '\n')
            if len(raw_question) < 2:
                break
            question = clear_line(raw_question, STOPWORDS)
            question = split_words(question)
            new_bow = dictionary.doc2bow(question)
            new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])
            most_sim_ids = get_most_similar_documents(new_doc_distribution,doc_topic_dist)
            index  = most_sim_ids[0]
            answer = convos_train[index][1]
            print('Bot: ', answer)
            
    
def dict_doc_score_(doc, KEY_LIST, sujet):
    
    """
    We return score of all one words and all two consequent words"""

    list_word_ = split_words(doc)    
    return {word: tf_idf_(word, doc, KEY_LIST, sujet) for word in list_word_}
   

def dict_doc_score_for_classif(doc, sujet): 
    """
    We return score of all one words and all two consequent words"""

    list_word_ = split_words(doc)    
    return {word: tf_idf_for_classif(word, doc, sujet) for word in list_word_}
    
def score_by_new_doc_(new_doc, doc, DICT):
    
    list_word_in_doc = split_words(new_doc)
    score = 0
    for word in list_word_in_doc:
            if word in DICT[doc]:
                 score += DICT[doc][word]
    return score



def train_lda(data, num_topics, chunksize, alpha, eta):
    """
    This function trains the lda model
    We setup parameters like number of topics, the chunksize to use in Hoffman method
    We also do 2 passes of the data since this is a small dataset, so we want the distributions to stabilize
    """
    dictionary = corpora.Dictionary(data)
    corpus = [dictionary.doc2bow(doc) for doc in data]
    t1 = time.time()
    # low alpha means each document is only represented by a small number of topics, and vice versa
    # low eta means each topic is only represented by a small number of words, and vice versa
    lda = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary,
                   alpha=alpha, eta=eta, chunksize=chunksize, minimum_probability=0.0, passes=2)
    t2 = time.time()
   # print("Time to train LDA model on ", len(data), "articles: ", int(t2-t1), "seconds")
    return dictionary, corpus, lda

def find_best_model(data):
    start = time.time()
    num_topics_best, chunk_best, alpha_best, eta_best = 5, 1, 1e-2, 0.5e-2
    dictionary, corpus, lda = train_lda(data, num_topics_best, chunk_best, alpha_best, eta_best)
    precision_best, recall_best, F_best = calcul_precision_recall_(train = True)
    num_topics = [5, 10, 20, 30, 60, 120, 240 ]
    chunksize = [128]
    alpha = [1e-2, 1e-1, 1, 1e+1]
    eta = [0.5e-2, 1e-2, 1e-1, 1, 1e+1]
    for num_topic in num_topics:
        for chunk in chunksize:
            for alpha_ in alpha:
                for eta_ in eta:
                    dictionary, corpus, lda = train_lda(data, num_topic, chunk, alpha_, eta_)
                    precision, recall, F = calcul_precision_recall_(train = True)
                    if F > F_best:
                        num_topic_best = num_topic
                        chunk_best = chunk
                        alpha_best = alpha_
                        eta_best = eta_
                        precision_best = precision
                        recall_best = recall
                        F_best = F
    print('Best result is precision {}, recall {} and F score {}'.format(\
                                precision_best, recall_best, F_best))
    print('Best parameters are num_topic {} chunk {}, alpha {} et eta {}'.format(\
                    num_topic_best, chunk_best, alpha_best, eta_best))
    print('Time ', time.time()-start)
                    
    
def get_all_words(convos):
    """
    prend tous les conversations dans corpus et renvoie un array de documents avec
    des vocabulaires séparés"""
    
    data = []
    for convo in convos:
        data.append(split_words(convo[0]))
    return data

def jensen_shannon(query, matrix):
    
    """
    This function implements a Jensen-Shannon similarity
    between the input query (an LDA topic distribution for a document)
    and the entire corpus of topic distributions.
    It returns an array of length M where M is the number of documents in the corpus
    """
    # lets keep with the p,q notation above
    p = query[None,:].T # take transpose
    q = matrix.T # transpose matrix
    m = 0.5*(p + q)
    return np.sqrt(0.5*(entropy(p,m) + entropy(q,m)))

def get_most_similar_documents(query, matrix, k=10):
    """
    This function implements the Jensen-Shannon distance above
    and returns the top k indices of the smallest jensen shannon distances
    """
    sims = jensen_shannon(query,matrix) # list of jensen shannon distances
    return sims.argsort()[:k] # the top k positional index of the smallest Jensen Shannon distan


def calcul_precision_recall_(train = False): 
    
    if not train:
        total_precision, total_recall = 0, 0
        len_ = len(convos_test)
        index_to_print = random.sample(range(len_), 30)
        for i in range(len(convos_test)):
            question = convos_test[i][0]
            question = split_words(question)
            new_bow = dictionary.doc2bow(question)
            new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])
            most_sim_ids = get_most_similar_documents(new_doc_distribution, doc_topic_dist)
            index = most_sim_ids[0]
            answer = convos_train[index][1]
            good_answer = convos_test[i][1]
            precision, recall = precision_recall\
            (answer, good_answer)
            total_precision  += precision
            total_recall     += recall
        total_precision = total_precision/len_
        total_recall = total_recall/len_      
        F = 2*total_precision*total_recall/(total_precision+total_recall)
       # print('ON TEST, WE HAVE: PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} '\
        #      .format(total_precision, total_recall,F))
        #print() 
    else:
        total_precision, total_recall = 0, 0
        len_ = len(convos_train)
        index_to_print = random.sample(range(len_), 30)
        for i in range(len(convos_train)):
            question = convos_train[i][0]
            question = split_words(question)
            new_bow = dictionary.doc2bow(question)
            new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])
            most_sim_ids = get_most_similar_documents(new_doc_distribution,doc_topic_dist)
            index = most_sim_ids[0]
            answer = convos_train[index][1]
            good_answer = convos_train[i][1]
            precision, recall = precision_recall\
            (answer, good_answer)
            total_precision  += precision
            total_recall     += recall
        total_precision = total_precision/len_
        total_recall = total_recall/len_      
        F = 2*total_precision*total_recall/(total_precision+total_recall)
      #  print('ON TRAIN, WE HAVE: PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} '\
       #       .format(total_precision, total_recall,F))
       # print()
    return total_precision, total_recall, F

def data_processed():
    """
    Traiter et retourner les conversations avec les questions bien nettoyées
    les questions sans nettoyer et le couple pour test
    """
    convos1, questions1 = get_all_convos(STOPWORDS,'data/trains_sujet.txt')
    convos2, questions2  = get_all_convos(STOPWORDS, 'data/trains_hors_sujet.txt')
    convos3, questions3 = get_all_convos(STOPWORDS, 'data/convos3septembre_train.txt')
    convos1.extend(convos2)
    convos1.extend(convos3)
    questions1.extend(questions2)
    questions1.extend(questions3)
    convos_test1, questions_test1 = get_all_convos(STOPWORDS, 'data/tests_16juillet.txt')
    convos_test2, questions_test2 = get_all_convos(STOPWORDS, 'data/convos3septembre_test.txt')
    convos_test1.extend(convos_test2)
    questions_test1.extend(questions_test2)
    index_suff = [i for i in range(len(convos1))]
    random.shuffle(index_suff)
    convos, questions = [], []
    for i in index_suff:
        convos.append(convos1[i])
        questions.append(questions1[i])
    index_suff_ = [i for i in range(len(convos_test1))]
    random.shuffle(index_suff_)
    convos_test, questions_test = [], []
    for i in index_suff_:
        convos_test.append(convos_test1[i])
        questions_test.append(questions_test1[i])    
    return convos, questions, convos_test, questions_test

if __name__ == '__main__':
    
    start = time.time()
    try:
        convos_train_
    except NameError:
        convos_train, questions, convos_test, questions_test = data_processed()
        data = get_all_words(convos_train)
        start = time.time()
        dictionary, corpus, lda = train_lda(data, 240, 128, 0.1, 0.005)
        doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in lda[corpus]])
        pr, recall, F = calcul_precision_recall_(train = False)
        print('Training on {} couples and test on {} couples'.\
             format(len(convos_train), len(convos_test)))
        print('on test, nous avons les scores sont pre {:.5f} recall {:.5f} et F {:.5f}'.format(pr, recall, F))
        pr, recall, F = calcul_precision_recall_(train = True)
        print('on train, nous avons, precision {:.5f} recall {:.5f} et F {:.5f}'.format(pr, recall, F))
        print('all time is ', int(time.time()-start), ' seconds')
        #Best parameters are num_topic 240 chunk 128, alpha 0.1 et eta 0.005

/home/ubuntu/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1303: RuntimeWarning: invalid value encountered in sqrt


Training on 2096 couples and test on 541 couples
on test, nous avons les scores sont pre 0.50906 recall 0.51397 et F 0.51150
on train, nous avons, precision 0.64109 recall 0.64615 et F 0.64361
all time is  66  seconds


In [348]:
for j in range(10):
    i = random.choice([i for i in range(100)])
    print(convos_test[i][0], '-----',questions_test[i])

modifi chaqu comet ----- On en change pas à chaque comète 
quel nombr voitur disponibl ----- quel est le nombre de véhicules disponibles
ailleur ----- Qui c'est, d'ailleurs 
jour tabl rond ----- C'est jour de table ronde 
pouv aid ----- vous pouvez m'aider
manqu truc ----- Il manque pas un truc 
sembl as problem km ----- il semble avoir un problème avec le kilométrage
quel voitur vitess plus faibl ----- quels véhicules ont vitesse plus faible
où où ----- D'où venez-vous ? Où allez-vous 
quel consomm moyen voitur ----- quelle est la consommation moyenne de mon véhicule


In [431]:
dictionary, corpus, lda = train_lda(data, 240, 128, 0.1, 0.005)
doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in lda[corpus]])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [3]:
convo = random.choice(convos_train)
question = convo[0]
print(question)
question = split_words(question)
print(question)
new_bow = dictionary.doc2bow(question)
print(new_bow)

nombr voitur progressent rout
['nombr', 'voitur', 'progressent', 'rout', 'nombr voitur', 'voitur progressent', 'progressent rout', 'nombr voitur progressent', 'voitur progressent rout']
[(71, 1), (257, 1), (258, 1), (681, 1), (5319, 1), (5320, 1), (5321, 1), (5322, 1), (5323, 1)]


In [1]:
dictionary, corpus, lda = train_lda(data, 240, 128, 0.1, 0.005)
doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in lda[corpus]])
def calcul_precision_recall_(train = False): 
    
    if not train:
        total_precision, total_recall = 0, 0
        len_ = len(convos_test)
        index_to_print = random.sample(range(len_), 30)
        for i in range(len(convos_test)):
            question = convos_test[i][0]
            question = split_words(question)
            new_bow = dictionary.doc2bow(question)
            new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])
            most_sim_ids = get_most_similar_documents(new_doc_distribution, doc_topic_dist)
            index = most_sim_ids[0]
            answer = convos_train[index][1]
            good_answer = convos_test[i][1]
            precision, recall = precision_recall\
            (answer, good_answer)
            total_precision  += precision
            total_recall     += recall
        total_precision = total_precision/len_
        total_recall = total_recall/len_      
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('ON TEST, WE HAVE: PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} '\
              .format(total_precision, total_recall,F))
        print() 
    else:
        total_precision, total_recall = 0, 0
        len_ = len(convos_train)
        index_to_print = random.sample(range(len_), 30)
        for i in range(len(convos_train)):
            question = convos_train[i][0]
            question = split_words(question)
            new_bow = dictionary.doc2bow(question)
            new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])
            most_sim_ids = get_most_similar_documents(new_doc_distribution, doc_topic_dist)
            index = most_sim_ids[0]
            answer = convos_train[index][0]
            good_answer = convos_train[i][0]
            precision, recall = precision_recall\
            (answer, good_answer)
            total_precision  += precision
            total_recall     += recall
        total_precision = total_precision/len_
        total_recall = total_recall/len_      
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('ON TRAIN, WE HAVE: PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} '\
              .format(total_precision, total_recall,F))
        print()
    return total_precision, total_recall, F
calcul_precision_recall_(train = True)
calcul_precision_recall_(train = False)

NameError: name 'train_lda' is not defined

In [257]:
def print_best_result():
        dictionary, corpus, lda = train_lda(data,700, 1000, 0.1, 0.0001)
        doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in lda[corpus]])
        pr, recall, F = calcul_precision_recall_(train = False)
        print('on test, nous avons, precision {:.5f} recall {:.5f} et F {:.5f}'.format(pr, recall, F))
        pr, recall, F = calcul_precision_recall_(train = True)
        print('on train, nous avons, precision {:.5f} recall {:.5f} et F {:.5f}'.format(pr, recall, F))
        #print('all time is ', int(time.time()-start), ' seconds')
print_best_result()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:582: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1298: RuntimeWarning: invalid value encountered in sqrt


on test, nous avons, precision 0.47672 recall 0.57861 et F 0.52275
on train, nous avons, precision 0.47173 recall 0.55623 et F 0.51051


In [280]:
print(len(doc_topic_dist))
print(random.choice(doc_topic_dist))

2096
[0.01666665 0.0166666  0.01666674 0.01666714 0.01666662 0.01666693
 0.01666654 0.01666683 0.01666686 0.01666637 0.01666689 0.01666704
 0.01666643 0.01666656 0.01666632 0.01666691 0.01666651 0.01666643
 0.01666642 0.01666675 0.01666659 0.01666681 0.01666705 0.01666684
 0.0166662  0.01666686 0.01666665 0.01666643 0.0166667  0.01666659
 0.01666643 0.01666706 0.01666638 0.01666692 0.01666666 0.01666727
 0.01666697 0.01666685 0.016667   0.01666671 0.0166666  0.01666657
 0.01666657 0.0166665  0.0166667  0.01666621 0.01666674 0.01666683
 0.01666649 0.01666634 0.01666657 0.01666677 0.01666638 0.01666619
 0.0166668  0.01666672 0.01666688 0.01666623 0.01666646 0.01666695]


In [ ]:
chat_()

This will be LDA method....
Bonjour, le bot d'Avicen à ton écoute
You: hello
Bot:  Vous savez bien.
You: salut
Bot:  Vous savez bien.
You: comment ça va
Bot:  ça va merci
You: je cherche ma voiture


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1303: RuntimeWarning: invalid value encountered in sqrt


Bot:  #ask_id# #problème#
You: bon, je veux ma voiture
Bot:  #ask_id# #position#
You: où est ma femme
Bot:  Partie faire un tour.
You: où sont les enfants
Bot:  Non. Bonne nuit.
You: à plus tard
Bot:  #ask_id# #moteur_coupé#
You: à tout à l'heure
Bot:  Non, pas ça !
You: pourquoi
Bot:  Il faudrait que je me décide ?
You: je veux un renseignement
Bot:  je suis là pour t'aider
You: où est la voiture
Bot:  ton véhicule est à #position#
You: et le roi est où
Bot:  J'en sais rien.
You: ta femme
Bot:  Il ferait beau voir.
